In [45]:
import json
import random

input_file = "billsum_v4_1/ca_test_data_final_OFFICIAL.jsonl"
dataset = [json.loads(line.strip()) for line in open(input_file)]
print(dataset[0]['summary'])
text = dataset[0]['summary']
# dataset = dataset[:10]

Existing property tax law establishes a veterans’ organization exemption under which property is exempt from taxation if, among other things, that property is used exclusively for charitable purposes and is owned by a veterans’ organization. This bill would provide that the veterans’ organization exemption shall not be denied to a property on the basis that the property is used for fraternal, lodge, or social club purposes, and would make specific findings and declarations in that regard. The bill would also provide that the exemption shall not apply to any portion of a property that consists of a bar where alcoholic beverages are served. Section 2229 of the Revenue and Taxation Code requires the Legislature to reimburse local agencies annually for certain property tax revenues lost as a result of any exemption or classification of property for purposes of ad valorem property taxation. This bill would provide that, notwithstanding Section 2229 of the Revenue and Taxation Code, no appro

In [46]:
from transformers import pipeline
from tqdm import tqdm

# 1. Initialize the pipeline
classifier = pipeline(
    "text-classification", 
    model="AbdullahBarayan/ModernBERT-base-doc_en-Cefr",
    device=0,
    top_k=None,
)

# 2. Define your texts
texts = [instance['summary'] for instance in dataset]

# 3. Run batch inference
# Setting batch_size here helps with memory management for large datasets
results = classifier(texts, batch_size=16)

assert len(results) == len(dataset)

# 4. Inspect results
for idx in tqdm(range(len(dataset))):
    text_result = results[idx]
    dataset[idx]['cefr_labels'] = text_result
    

Device set to use cuda:0
100%|██████████| 1237/1237 [00:00<00:00, 1330688.39it/s]


In [47]:
output_file = "billsum_v4_1/ca_test_data_final_OFFICIAL_with_cefr_labels.jsonl"

with open(output_file, "w") as fp:
    for instance in dataset:
        fp.write(json.dumps(instance).strip() + "\n")

In [55]:
from collections import Counter

input_file = "billsum_v4_1/org_with_cefr_labels/ca_test_data_final_OFFICIAL.jsonl"
dataset = [json.loads(line.strip()) for line in open(input_file)]

label_dist = []
for instance in dataset:
    cefr_labels = instance["cefr_labels"]
    cefr_labels = [(cefr["score"], cefr["label"]) for cefr in instance["cefr_labels"]]
    cefr_labels = sorted(cefr_labels, reverse=True)
    label_dist.append(cefr_labels[0][1])

print(Counter(label_dist))


Counter({'B2': 1139, 'C1': 80, 'B1': 15, 'A2': 3})
